<a href="https://colab.research.google.com/github/DLHTurboDieselLtd/Account_class_exercise/blob/main/Transformed_data_production.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import re
import pandas as pd
import numpy as np
import glob
from google.colab import files

In [ ]:
def create_df(filename = '/content/drive/MyDrive/combined/*.csv', save = False):
  # read in csv file
  df = pd.concat(map(pd.read_csv, glob.glob(filename)))

  # truncate machine desc
  df['Machine'] = [re.search(r'(\w{3})\d*-UP', machine_name) for machine_name in df['Machine']]

  df['Engine'] = [re.search(r"POWERED BY\s*([\w\-\.]+)\s*(?:Engine|ENGINE)", machine_name, re.IGNORECASE) for machine_name in df['Machine']]

  df['Machine'] = f"{df['Machine']}_{df['Engine']}"

  # process description col
  df['Description'] = df['Description'].apply(str)
  df['Description'] = [s.replace(" ", '-') for s in df['Description']]
  df['Description'] = [re.sub(r'^-|-$', '', s) for s in df['Description']]

  # keep only certain cols
  df = df.iloc[:, [1,3,4]].drop_duplicates()

  # copy df
  df = df.copy()

  # create item col
  df['Item'] = df['Part numbers'] + ' ' +	df['Description']

  # save to csv
  if save == True:
    df.to_csv('create_df.csv', index=False)
    files.download('create_df.csv')
  else:
    pass

  return df

In [ ]:
df_combined = create_df()
df_combined

In [ ]:
def get_combined_groups(df, group='Item', save = False):
  # groupby 'Item'
  df_combined_gbo = df.groupby([group])

  # convert gbo to list
  l = list(df_combined_gbo)

  # convert gbo to array
  l_array = np.array(l, dtype=object)

  # create containers for data
  machine_container = []
  item_container = []

  # interate through the array
  for i in range(l_array.shape[0]):
    machine_container.append(list(set(l_array[i][1]['Machine'].values)))

  for i in range(l_array.shape[0]):
    item_container.append(l_array[i][0])

  # create DataFrame
  machine_df = pd.DataFrame({'Machine':list(map(sorted, machine_container))})
  item_df = pd.DataFrame({'Item':item_container})

  # combine series
  df_combined_final = pd.concat([item_df, machine_df], axis=1)

  # split the Item col
  df_combined_final[['Part Number', 'Description']] = df_combined_final['Item'].str.split(' ', 1, expand = True)

  # subset the final df
  df_combined_final = df_combined_final.iloc[:, [2,3,1]]

  df_combined_final['Machine'] = pd.DataFrame({'Machine':list(map('\r\n'.join, df_combined_final['Machine']))})
  df_combined_final['Description'] = [x.replace(',', '') for x in df_combined_final['Description']]

  # save to csv
  if save == True:
    df_combined_final.to_csv('df_combined_final.csv', index=False)
    files.download('df_combined_final.csv')
  else:
    pass

  return df_combined_final

In [ ]:
# np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)

df = get_combined_groups(df_combined, save = True)
df

In [ ]:
# df = pd.read_csv(filepath_or_buffer='/content/drive/MyDrive/combined/CAT_Grouped.csv')
# df

In [ ]:
df = df.iloc[0:100]
df

In [ ]:
df.nunique()

In [ ]:
df.to_csv('CAT_Grouped_23.02.25.csv', index=False)
files.download('CAT_Grouped_test.23.02.25.csv')

In [ ]:
1589554 / 5

In [ ]:
!pip install xlsxwriter

In [ ]:
df1 = df.iloc[0:317910]
df2 = df.iloc[317910:(317910+317910)]
df3 = df.iloc[(317910+317910):(317910+317910+317910)]
df4 = df.iloc[(317910+317910+317910):(317910+317910+317910+317910)]
df5 = df.iloc[(317910+317910+317910+317910):(317910+317910+317910+317910+317910)]

writer = pd.ExcelWriter('CAT_multiple.xlsx', engine='xlsxwriter')

df1.to_excel(writer, sheet_name='Sheet1', index=False)
df2.to_excel(writer, sheet_name='Sheet2', index=False)
df3.to_excel(writer, sheet_name='Sheet3', index=False)
df4.to_excel(writer, sheet_name='Sheet4', index=False)
df5.to_excel(writer, sheet_name='Sheet5', index=False)

writer.save()
files.download('CAT_multiple.xlsx')